In [0]:
import sys
sys.path.append("/content/drive/My Drive/snakeAI")

from collections import namedtuple

Experience = namedtuple('Experience', ('state', 'direction', 'next_state', 'reward'))

import tensorflow as tf
import tensorflow.keras as k

from Snake.environment import SnakeMaze
from Snake.variables import Status, Cell
from models.replay_memory import ReplayMemory
from models.model_train import train_dqn
from Snake.utils import euclidean_distance

from datetime import datetime
import numpy as np
import cv2
import os
from time import time


# ResNet50

In [0]:
def ResNet50(input_shape, num_not_trainable_blocks=4):
  base_model = k.applications.ResNet50(include_top=False, input_shape=input_shape)
  base_model.trainable = False

  for l in base_model.layers:
    if l.name.split("_")[0] == f"conv{num_not_trainable_blocks+1}":
      break
    l.trainable = True

  model = k.Sequential([
      base_model,
      k.layers.Flatten(),
      k.layers.Dense(4096, activation='relu'),
      k.layers.Dense(2048, activation='relu'),
      k.layers.Dense(1024, activation='relu'),
      k.layers.Dense(512, activation='relu'),
      k.layers.Dense(4, activation='linear'),
  ])

  model.build(input_shape=input_shape)
  return model

### Configuration

In [3]:
"""
            Training model

            :param model: tf.keras.Model
                An instance ovr the keras.Model
            :param reward: function
                A function that returns the reward of the current state

                Args:
                    ::param snake: Snake
                    ::param env: SnakeMaze
                    ::param direction: Direction

                Returns:
                    int: The current reward

            :param optimizer: tf.keras.Optimizer
                The optimizer used to train the model

            :param kwargs:
                save_images (bool): False
                    If true images are saved on the eval episode

                save_videos (bool): False
                    If true videos are saved on the eval episode

                save_models (bool): False
                    If true models are saved on the eval episode

                save_graphs (bool): False
                    If true graphs are saved on the eval episode

                evaluate_each (int): 50
                    Number of episodes between two evaluations

                num_rolling_avg_sample (int): 50
                    Number of episodes that are used for the rolling average

                max_steps_per_episode (int): 200
                    Max number of steps in each episode

                num_episodes (int, None): None
                    Number of episodes to train, if None runs forever

                gamma (float): .8
                    The discount factor

                epsilon (float): 1.
                    The exploration/exploitation rate

                epsilon_decay (float): 0.0005
                    The decay of the exploration/exploitation rate

                min_epsilon (float): 0.01
                    The epsilon convergence

                memory_size (int): 100000
                    Capacity of the reply memory, if less then 50 Replay Memory will not be used

                boundaries (bool): True
                    Include boundaries in the maze

                maze_width (int): 10
                    The width of the maze

                maze_height (int, None): None
                    The height of the maze, if not specified the maze is squared

                max_snakes (int): 1
                    The max number of snakes in the environment

                path_to_weights (str): None
                    Path to saved weights for the model

                image_size (tuple[int, int]): (112, 112)
                    The size of the image used as input in the model

                verbose (bool): True
                    Print info about the training

                training_dir (str): './'
                    Path to the directory for training
            """

"\n            Training model\n\n            :param model: tf.keras.Model\n                An instance ovr the keras.Model\n            :param reward: function\n                A function that returns the reward of the current state\n\n                Args:\n                    ::param snake: Snake\n                    ::param env: SnakeMaze\n                    ::param direction: Direction\n\n                Returns:\n                    int: The current reward\n\n            :param optimizer: tf.keras.Optimizer\n                The optimizer used to train the model\n\n            :param kwargs:\n                save_images (bool): False\n                    If true images are saved on the eval episode\n\n                save_videos (bool): False\n                    If true videos are saved on the eval episode\n\n                save_models (bool): False\n                    If true models are saved on the eval episode\n\n                save_graphs (bool): False\n                   

In [0]:
NUM_NOT_TRAINABLE_BLOCKS = 4

IMAGE_SIZE = (64, 64)
IMAGE_SHAPE = (*IMAGE_SIZE, 3)

learning_rate = 1e-5

config = {
  "num_rolling_avg_sample": 10,
  "evaluate_each": 20,
  "save_models": True,
  "save_graphs": True,
  "gamma": .99,
  "epsilon_decay": 0.0005,
  "boundaries": False,
  "maze_width": 10,
  "image_size": IMAGE_SIZE,
  "batch_size": 128,
  "comment": f"""
  Architecture: ResNet50
  Weights: imagenet, {NUM_NOT_TRAINABLE_BLOCKS} non trainable blocks
  Optimizer: Adam ( with amsgrad )
  Hyperparameters:
    lr = {learning_rate}
  Reward:
      if snake.status == Status.DEAD:
        return 0
    else:
        r = 0
        r += 1000 if snake.steps_without_food == 1 else 0
        return r

  Reward only for eating food
  """
}

### Training

In [0]:
envv = SnakeMaze(10, 10, 1)
envv.reset()

In [0]:
def reward(snake, env: SnakeMaze, direction):
    if snake.status == Status.DEAD:
        return -1000
        return 0
    else:
        r = 0
        r -= euclidean_distance(envv.snakes[0].body[0], list(envv.food)[0])
        r += 50 if snake.steps_without_food == 1 else 0
        return r

In [0]:
t = time()
config["training_dir"] = os.path.join(*['drive', 'My Drive', 'snakeAI', 'trainings', 'transfer_learningDQN', "ResNet50", datetime.now().strftime('%d%h%Y__%H%M%S%f')])
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)

model = ResNet50(input_shape=IMAGE_SHAPE, num_not_trainable_blocks=NUM_NOT_TRAINABLE_BLOCKS)
model.build(input_shape=(None, *IMAGE_SHAPE))
model.compile(optimizer=optimizer)

model = train_dqn(model, optimizer, reward, **config)
t = time() - t

94773248/94765736 [==============================] - 2s 0us/step
____________________________________________________________________________________________
Episode 1 Done!
Episode reward: -500.0
Epsilon: 1.0
Replay Memory size: 136

Model saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/model
Graph saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/Reward.png
____________________________________________________________________________________________
Episode 2 Done!
Episode reward: -550.0
Epsilon: 0.9995
Replay Memory size: 176

____________________________________________________________________________________________
Episode 3 Done!
Episode reward: -550.0
Epsilon: 0.9990000000000001
Replay Memory size: 280

____________________________________________________________________________________________
Episode 4 Done!
Episode reward: -500.0
Epsilon: 0.9985000000000002
Replay Memory size: 320


/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/Reward.png
____________________________________________________________________________________________
Episode 402 Done!
Episode reward: -500.0
Epsilon: 0.7995000000000221
Replay Memory size: 254

____________________________________________________________________________________________
Episode 403 Done!
Episode reward: -1173.0
Epsilon: 0.7990000000000221
Replay Memory size: 250

____________________________________________________________________________________________
Episode 404 Done!
Episode reward: -500.0
Epsilon: 0.7985000000000222
Replay Memory size: 258

____________________________________________________________________________________________
Episode 405 Done!
Episode reward: -550.0
Epsilon: 0.7980000000000222
Replay Memory size: 330

____________________________________________________________________________________________
Episode 406 Done!
Episode reward: -550.0
Eps

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/Reward.png
____________________________________________________________________________________________
Episode 422 Done!
Episode reward: -550.0
Epsilon: 0.7895000000000232
Replay Memory size: 356

____________________________________________________________________________________________
Episode 423 Done!
Episode reward: -500.0
Epsilon: 0.7890000000000232
Replay Memory size: 332

____________________________________________________________________________________________
Episode 424 Done!
Episode reward: -550.0
Epsilon: 0.7885000000000233
Replay Memory size: 244

____________________________________________________________________________________________
Episode 425 Done!
Episode reward: -450.0
Epsilon: 0.7880000000000233
Replay Memory size: 252

____________________________________________________________________________________________
Episode 426 Done!
Episode reward: -1497.0
Eps

...

Graph saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/Reward.png
____________________________________________________________________________________________
Episode 662 Done!
Episode reward: -600.0
Epsilon: 0.6695000000000364
Replay Memory size: 248

____________________________________________________________________________________________
Episode 663 Done!
Episode reward: -550.0
Epsilon: 0.6690000000000365
Replay Memory size: 320

____________________________________________________________________________________________
Episode 664 Done!
Episode reward: -550.0
Epsilon: 0.6685000000000365
Replay Memory size: 264

____________________________________________________________________________________________
Episode 665 Done!
Episode reward: -450.0
Epsilon: 0.6680000000000366
Replay Memory size: 272

____________________________________________________________________________________________
Episode 666 Done!
Episode reward: -1407.0
Eps

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/Reward.png
____________________________________________________________________________________________
Episode 682 Done!
Episode reward: -1407.0
Epsilon: 0.6595000000000375
Replay Memory size: 301

____________________________________________________________________________________________
Episode 683 Done!
Episode reward: -600.0
Epsilon: 0.6590000000000376
Replay Memory size: 341

____________________________________________________________________________________________
Episode 684 Done!
Episode reward: -550.0
Epsilon: 0.6585000000000376
Replay Memory size: 285

____________________________________________________________________________________________
Episode 685 Done!
Episode reward: -550.0
Epsilon: 0.6580000000000377
Replay Memory size: 229

____________________________________________________________________________________________
Episode 686 Done!
Episode reward: -500.0
Eps

/content/drive/My Drive/snakeAI/Snake/utils.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(2, 1)


Graph saved at: drive/My Drive/snakeAI/trainings/transfer_learningDQN/ResNet50/10Jun2020__200157933012/Reward.png
____________________________________________________________________________________________
Episode 702 Done!
Episode reward: -500.0
Epsilon: 0.6495000000000386
Replay Memory size: 317

____________________________________________________________________________________________
Episode 703 Done!
Episode reward: -500.0
Epsilon: 0.6490000000000387
Replay Memory size: 325

____________________________________________________________________________________________
Episode 704 Done!
Episode reward: -550.0
Epsilon: 0.6485000000000387
Replay Memory size: 269

____________________________________________________________________________________________
Episode 705 Done!
Episode reward: -1470.0
Epsilon: 0.6480000000000388
Replay Memory size: 300

____________________________________________________________________________________________
Episode 706 Done!
Episode reward: -550.0
Eps

In [0]:
assert False
env = SnakeMaze(20, 20, 1, with_boundaries=False)
env.reset()
imgs = []
while env.num_active_agents:
    imgs.append(env.snake_matrices[0])
    # state = resize_image(env.snake_matrices[0], IMAGE_SIZE)
    direction = np.argmax(model(np.array([state])))
    env.step({0: direction})

In [0]:
generate_animation(imgs)